In [3]:
from tensorflow.keras import utils
#from google.colab import files
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow.keras as keras

In [4]:
df = pd.read_csv("creditcard.csv")

In [5]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [6]:
data = df.drop(['Time'], axis=1) # Удаляем столбец со временем

data['Amount'] = StandardScaler().fit_transform(
    data['Amount'].values.reshape(-1, 1)) # Нормализуем столбец с суммой транзакции

frauds = data[data.Class == 1]
normal = data[data.Class == 0]

In [7]:
RANDOM_SEED = 42
X_train, X_test = train_test_split(normal, test_size=0.2, random_state=RANDOM_SEED)

X_train = X_train.drop(['Class'], axis=1)
X_test = pd.concat([X_test, frauds]) # Добавляем все мошеннические транзакции в тестовый набор данных

y_test = X_test['Class'] # Делаем метки для тестового набора данных
X_test = X_test.drop(['Class'], axis=1) # Удаляем класс у тестового набора данных

X_train = X_train.values # Преобразуем данные в массивы numpy
X_test = X_test.values # Преобразуем данные в массивы numpy

In [19]:
def creditcard_autoencoder():

    input_ = Input(shape=(X_train.shape[1], )) # входной слой
    x = Dense(10, activation='relu')(input_)
    x = Dense(X_train.shape[1], activation='relu')(x) # выходной слой

    autoencoder = Model(inputs=input_, outputs=x)
    autoencoder.compile(optimizer='Adam', loss='mse')
    return autoencoder

model = creditcard_autoencoder()
history = model.fit(X_train, X_train, epochs=20, batch_size=32)

Epoch 1/20
7108/7108 [==============================] - 6s 689us/step - loss: 0.8679
Epoch 2/20
7108/7108 [==============================] - 6s 806us/step - loss: 0.6699
Epoch 3/20
7108/7108 [==============================] - 5s 685us/step - loss: 0.6584 0s - loss: 0.658
Epoch 4/20
7108/7108 [==============================] - 5s 658us/step - loss: 0.6797
Epoch 5/20
7108/7108 [==============================] - 5s 675us/step - loss: 0.6574
Epoch 6/20
7108/7108 [==============================] - 5s 679us/step - loss: 0.6625
Epoch 7/20
7108/7108 [==============================] - 5s 649us/step - loss: 0.6557
Epoch 8/20
7108/7108 [==============================] - 5s 667us/step - loss: 0.6517
Epoch 9/20
7108/7108 [==============================] - 5s 691us/step - loss: 0.6610 0s 
Epoch 10/20
7108/7108 [==============================] - 5s 739us/step - loss: 0.6530
Epoch 11/20
7108/7108 [==============================] - 6s 850us/step - loss: 0.6571
Epoch 12/20
7108/7108 [===================

In [20]:
predictions = model.predict(X_test) # делаем предсказания

In [22]:
mse = np.mean(np.power(X_test - predictions, 2), axis=1) # MSE автокодировщика

mse_normal = mse[y_test.values == 0] # ошибка в нормальных транзакциях
mse_frauds = mse[y_test.values == 1] # ошибка в мошеннических транзакциях

print("Минимальная ошибка нормальных трансакций:", round(min(mse_normal),4))
print("Максимальная ошибка нормальных трансакций:", round(max(mse_normal),4))
print("Средняя ошибка нормальных трансакций:", round(sum(mse_normal) / len(mse_normal),4))

print("Минимальная ошибка мошеннических трансакций:", round(min(mse_frauds),4))
print("Максимальная ошибка мошеннических трансакций:", round(max(mse_frauds),4))
print("Средняя ошибка мошеннических трансакций:", round(sum(mse_frauds) / len(mse_frauds),4))

Минимальная ошибка нормальных трансакций: 0.0451
Максимальная ошибка нормальных трансакций: 1291.6608
Средняя ошибка нормальных трансакций: 0.6686
Минимальная ошибка мошеннических трансакций: 0.1052
Максимальная ошибка мошеннических трансакций: 272.4461
Средняя ошибка мошеннических трансакций: 25.6185


In [23]:
def getAccByBias(bias):
    isNormal = mse_normal < bias
    isFrauds = mse_frauds > bias

    accNormal = sum(isNormal) / len(isNormal)
    accFaruds = sum(isFrauds) / len(isFrauds)

    print("Распознано нормальных трансакций: ", round(100*accNormal), "%", sep="")
    print("Распознано мошеннических трансакций: ", round(100*accFaruds), "%", sep="")
    print("Средняя accuracy распознавания: ", round(50*(accNormal + accFaruds)), "%", sep="")

In [24]:
getAccByBias(2.5)

Распознано нормальных трансакций: 97%
Распознано мошеннических трансакций: 83%
Средняя accuracy распознавания: 90%
